In [1]:
import os
import io
import sys
cwd = os.getcwd()
sys.path.insert(0, cwd)
from module.importData import importData
from module.toWordList import toWordList
from module.steamingWiki import steamingWiki
from module.makeModelGensim import makeModelGensim
from module.toVectore import toVectore
#from module.modelLSTM import modelLSTM
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, cohen_kappa_score
import pandas as pd
import numpy as np
import gensim
import keras
from keras.models import load_model

C:\Users\s2\Anaconda3\envs\tgpu\lib\site-packages\smart_open\ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')
C:\Users\s2\Anaconda3\envs\tgpu\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [2]:
np.set_printoptions(threshold=sys.maxsize)
pd.get_option("display.max_rows", 1000)
pd.set_option('display.max_columns', 1000)

In [3]:
wikiSource			= 'enwiki'
answerData			= 'aesASAD3.csv'
questionData		= 'qes.csv'
dirData				= cwd+'/data/'
corpusInput			= wikiSource+'.bz2'
wikiOutput			= wikiSource+'.txt'
fileExtension		= 'bin'
trainingAlgoritm	= 0
numDimension		= 200
modelOutput			= wikiSource+'_word2vec_'+str(numDimension)+'_'+str(trainingAlgoritm)+'.'+fileExtension

In [4]:
dAnswer, dQuestion = importData(answer= dirData+answerData, question= dirData+questionData).openData()

if not(os.path.exists(dirData+modelOutput)):
	if not(os.path.exists(dirData+wikiOutput)):
		steamingWiki(corpusInput=corpusInput, wikiOutput=wikiOutput).execute()
	makeModelGensim(wikiOutput=wikiOutput, modelOutput=modelOutput, numDimension=numDimension, trainingAlgoritm=trainingAlgoritm).execute()

if fileExtension != 'bin':
	model = gensim.models.word2vec.Word2Vec.load(dirData+modelOutput)
else:
	model = gensim.models.KeyedVectors.load_word2vec_format(dirData+modelOutput, unicode_errors='ignore')


In [5]:
crossVal = KFold(n_splits=2, random_state=True, shuffle=True)

count = 1

In [6]:
dAnswer.head()

,Essay_id,Answer,Score,TrueAnswer
0,1,Some additional information that we would need...,1,In order to replace the experiment yal would n...
1120,1,"After reading the groups procedure, the additi...",2,In order to replace the experiment yal would n...
1119,1,Find the difference in mass.,0,In order to replace the experiment yal would n...
1118,1,One piece of additional information that I wou...,2,In order to replace the experiment yal would n...
1117,1,Step 1 should be to gather each materials. ^p ...,0,In order to replace the experiment yal would n...


In [7]:
xtrain = dAnswer.loc[:,['Answer','TrueAnswer']]

In [8]:
xtrain.head()

,Answer,TrueAnswer
0,Some additional information that we would need...,In order to replace the experiment yal would n...
1120,"After reading the groups procedure, the additi...",In order to replace the experiment yal would n...
1119,Find the difference in mass.,In order to replace the experiment yal would n...
1118,One piece of additional information that I wou...,In order to replace the experiment yal would n...
1117,Step 1 should be to gather each materials. ^p ...,In order to replace the experiment yal would n...


In [9]:
#keras.utils.to_categorical(dAnswer.loc[:,['Score']], num_classes=6)

In [10]:
class modelLSTM(object):
	from keras.models import Sequential, Model
	from keras.layers import LSTM, Dense, Dropout, Bidirectional, Lambda, Dot, Subtract, Multiply
	import keras.backend as K
	from keras import optimizers

	def biSiamenseModel(self, inputD, distance=False):

		from keras.layers import Input, concatenate, subtract, dot
		from keras import backend as K

		def euclidean_distance(vects):
			x, y = vects
			sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
			return K.sqrt(K.maximum(sum_square, K.epsilon()))
        
		def cosine_distance(vests):
			x, y = vests
			x = K.l2_normalize(x, axis=-1)
			y = K.l2_normalize(y, axis=-1)
			return -K.mean(x * y, axis=-1, keepdims=True)

		def manhattan_distance(vects):
			x, y = vects
			return K.sum((K.abs(x - y)), axis=1, keepdims=True)

		def exponent_neg_manhattan_distance(vects):
			x, y = vects
			return K.exp(-K.sum(K.abs(x-y), axis=1, keepdims=True))


		MA = Input(shape= inputD, dtype="float32")
		MB = Input(shape= inputD, dtype="float32")

		x = self.Bidirectional(self.LSTM(150, dropout=0.2, recurrent_dropout=0.2, return_sequences=True), input_shape=inputD)(MA)
		x = self.Bidirectional(self.LSTM(75, dropout=0.2, recurrent_dropout=0.2))(x)

		y = self.Bidirectional(self.LSTM(150, dropout=0.2, recurrent_dropout=0.2, return_sequences=True), input_shape=inputD)(MB)
		y = self.Bidirectional(self.LSTM(75, dropout=0.2, recurrent_dropout=0.2))(y)
		# a2 = (self.Lambda(function=lambda a: euclidean_distance(a),output_shape=lambda a: (a,1)))([x,y])

		if distance:
			a2 = (self.Lambda(function=lambda a: cosine_distance(a),output_shape=lambda a: (a,1)))([x,y])
		else:
			#a2 = concatenate([x,y])
			a2 = self.Multiply()([x,y])
		
		out = self.Dense(1, activation='relu')(a2)

		model = self.Model(inputs=[MA, MB], outputs=out)
		sgd = self.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
		model.compile(loss='mean_squared_error', optimizer=sgd, metrics=['mae'])

		model.summary()

		return model	

In [11]:
for dx, dy in crossVal.split(dAnswer):

	trainSAnswer = []
	trainTAnswer = []
	testSAnswer = []
	testTAnswer = []
	
	print("\n--------Fold {}--------\n".format(count))
	train, test= dAnswer.iloc[dx], dAnswer.iloc[dy]
	
	
	xtrain = train.loc[:,['Answer', 'TrueAnswer']]
	xtest = test.loc[:,['Answer', 'TrueAnswer']]
	ytrain = train.loc[:,['Score']].values
	ytest = test.loc[:,['Score']].values
	#ytraincat, idtrain = np.unique(train.loc[:,['Score']].values, return_inverse=True)
	#ytrain = keras.utils.to_categorical(idtrain, len(ytraincat))
	#ytestcat, idtest = np.unique(test.loc[:,['Score']].values, return_inverse=True)
	#ytest = keras.utils.to_categorical(idtest, len(ytestcat))
	#ytrain = keras.utils.to_categorical(train.loc[:,['Score']].values, num_classes=6)
	#ytest = keras.utils.to_categorical(test.loc[:,['Score']].values, num_classes=6)
	
	# bagian ini biar universal aja.
	
	[trainSAnswer.append(toWordList().sentenceToWordList(a1[0], changeNumber2Word= True)) for a1 in xtrain.loc[:,['Answer']].values]
	[trainTAnswer.append(toWordList().sentenceToWordList(a1[0], changeNumber2Word= True)) for a1 in xtrain.loc[:,['TrueAnswer']].values]
	[testSAnswer.append(toWordList().sentenceToWordList(a1[0], changeNumber2Word= True)) for a1 in xtest.loc[:,['Answer']].values]
	[testTAnswer.append(toWordList().sentenceToWordList(a1[0], changeNumber2Word= True)) for a1 in xtest.loc[:,['TrueAnswer']].values]
	
	vtrainSAnswer, vtrainTAnswer = toVectore(essays = trainSAnswer, trueAnswer=trainTAnswer, model = model, numFeature= numDimension, average=False, distance=True).changeToVector()
	vtestSAnswer, vtestTAnswer = toVectore(essays = testSAnswer, trueAnswer=testTAnswer, model = model, numFeature= numDimension, average=False, distance=True).changeToVector()
	modelNetwork = modelLSTM().biSiamenseModel(inputD=(vtrainSAnswer.shape[1], vtrainSAnswer.shape[2]))
	modelNetwork.fit([vtrainSAnswer, vtrainTAnswer], ytrain, batch_size=50, epochs=10)
	pred = modelNetwork.predict([vtestSAnswer, vtestTAnswer])


--------Fold 1--------

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 180, 200)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 180, 200)     0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 180, 300)     421200      input_1[0][0]                    
_______________________________________________________________________________________________

In [12]:
dfa = pd.DataFrame()

In [13]:
pred

array([[2.24126792e+00],
       [1.01802492e+00],
       [0.00000000e+00],
       [2.17930031e+00],
       [1.89538753e+00],
       [5.03256202e-01],
       [1.23121977e-01],
       [2.07342172e+00],
       [1.53456461e+00],
       [2.35054302e+00],
       [2.04150653e+00],
       [1.87314951e+00],
       [5.20980716e-01],
       [2.24856424e+00],
       [1.90149844e-01],
       [0.00000000e+00],
       [5.34928441e-01],
       [2.46858072e+00],
       [2.27634287e+00],
       [2.48105931e+00],
       [2.22520065e+00],
       [2.18197703e+00],
       [1.92917967e+00],
       [1.82937431e+00],
       [4.95194882e-01],
       [2.20869493e+00],
       [1.30385256e+00],
       [2.21653891e+00],
       [2.22917557e+00],
       [1.51610720e+00],
       [1.25273383e+00],
       [1.06143522e+00],
       [2.36261749e+00],
       [7.83085942e-01],
       [1.81802261e+00],
       [2.29267931e+00],
       [2.23824978e+00],
       [2.24751449e+00],
       [9.02902126e-01],
       [0.00000000e+00],


In [14]:
cohen_kappa_score(ytest, np.around(pred), weights='quadratic')

0.7023250661104876

In [15]:
mean_squared_error(ytest, np.around(pred))

0.5330935251798561

In [16]:
mean_absolute_error(ytest, np.around(pred))

0.4347721822541966